In [13]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [26]:
import machine
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5), freq=45000)
print(i2c.scan())
print(i2c.readfrom_mem(0x69, 0x03, 2))


[]


Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
OSError: [Errno 110] ETIMEDOUT


In [11]:
print(i2c.readfrom_mem(0x69, 0x00, 2))


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: [Errno 19] ENODEV


In [63]:
%sendtofile --source BME280_funcs.py

Sent 79 lines (3304 bytes) to BME280_funcs.py.


In [88]:
from machine import I2C, Pin
#i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
import machine
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
print(i2c.scan())

from BME280_funcs import bme280init, readBME280
bme280init(i2c, 0x76)
print(readBME280(0x76))


[118]
(23.73, 62458.5, 69.042)


In [65]:
from BME280_funcs import bme280init, readBME280
bme280init(i2c, 0x76)
print(readBME280(0x76))


(23.73, 62458.5, 69.042)


In [70]:
import time
for i in range(10):
    time.sleep(0.5)
    print(readBME280(0x76))

(22.64, 99784.0, 22.9854)
(22.63, 99782.4, 23.0068)
(22.6, 99759.0, 22.9941)
(22.85, 100013.0, 22.9287)
(23.38, 100142.0, 22.7764)
(24.13, 100418.0, 22.624)
(24.82, 100484.0, 22.5127)
(25.01, 100379.0, 22.5889)
(24.99, 100121.0, 22.7236)
(24.39, 99859.6, 23.0303)


In [24]:
#print(i2c.scan())
from BME280_funcs import bme280init, readBME280    #77
bme280init(i2c)
#print(readBME280())

Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "BME280_funcs.py", line 14, in bme280init
OSError: [Errno 19] ENODEV


In [ ]:
# Some MQTT based main files for the esp8266 sensors in DoESLiverpool
# For cheaper deployment

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

In [11]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB8 --baud=115200 
Ready.


In [8]:
%ls

Listing directory '/'.
      228    boot.py
      116    config.txt
     3462    dmpConfig.txt
     1929    dmpMemory.bin
     3545    max7219.py
      333    testleds.py
       15    webrepl_cfg.py


In [9]:
%fetchfile --print config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.30.194
pinled       2
boardname    esp8266LOLIN

In [ ]:
# Code below is to run just the SI7021 humidity temp sensor
# (and then the dallas temperature string)

Sent 5 lines (112 bytes) to config.txt.


In [2]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
boardname    esp8266B

Sent 5 lines (104 bytes) to config.txt.


In [ ]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.30.194
pinled       2
boardname    esp8266LOLIN

In [ ]:
# we can move some of these state functions into another file

In [18]:
%sendtofile main.py

# SI7021 humidity and dallas string
import time, network, machine, ubinascii, esp

fconfig = dict(x.split()  for x in open("config.txt"))

pled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
plight = machine.ADC(0)

si = network.WLAN()
si.active(True)
si.connect(fconfig["wifiname"], fconfig["wifipassword"])
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print(ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

from umqtt.robust import MQTTClient
client = MQTTClient(fconfig["boardname"], fconfig["mqttbroker"])
client.connect()
topicip = fconfig["boardname"]+"/ip"
topicmac = fconfig["boardname"]+"/mac"
topicfreemem = fconfig["boardname"]+"/freemem"
macaddress = ubinascii.hexlify(si.config('mac'),':').decode().upper()

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)

# dallas string sensor code; G=Orange, D8=Yellow, 3V3=Blue
import onewire, ds18x20
pindallas = machine.Pin(13, machine.Pin.OUT, machine.Pin.PULL_UP)
dallasobj = 5 # tries
dallasscanned = [ ]
topicds = [ ]
def dallasinitconvert():
    global dallasobj, dallasscanned
    if isinstance(dallasobj, int):
        try:
            dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))
        except onewire.OneWireError as e:
            dallasobj -= 1
            return []
        dallasscanned = dallasobj.scan()
        print("dallasscanned", dallasscanned)
        if dallasscanned:
            sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
            dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
            print("DS18B20 n=%d"%len(dallasscanned))
            topicds.extend([ fconfig["boardname"]+"/DS18B20/"+str(i)  for i in range(len(dallasscanned)) ])
    if dallasscanned:
        dallasobj.convert_temp()

# SI7021 humidity sensor code
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
bsi7021init = False
def SI7021humiditytemp():
    global bsi7021init
    if not bsi7021init:
        i2c.writeto(0x40, b'\xFE')
        time.sleep_ms(20)
        bsi7021init = True
    i2c.writeto(0x40, b'\xE5')
    time.sleep_ms(20)   # give it time to take a reading or it fails
    bh = i2c.readfrom(0x40, 2)
    bt = i2c.readfrom_mem(0x40, 0xE0, 2)
    rh = (bh[0]<<8)+(bh[1]&0xFC)
    rt = (bt[0]<<8)+(bt[1]&0xFC)
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

topichumid = fconfig["boardname"]+"/SI7021/humid"
topictemp = fconfig["boardname"]+"/SI7021/temp"
topicldr = fconfig["boardname"]+"/LDR"

tdallasconverttime = 0
t0 = time.ticks_ms()
plightv0 = 0
tipnumbernext = 0
while True:
    t = time.ticks_ms()
    if t > tipnumbernext:
        client.publish(topicip, ipnumber, retain=True, qos=1)
        client.publish(topicmac, macaddress, retain=True, qos=1)
        client.publish(topicfreemem, "%d" % esp.freemem(), retain=True, qos=1)
        tipnumbernext = t + 60000
    
    plightv = plight.read()
    if abs(plightv - plightv0) > 45:
        client.publish(topicldr, "%d"%plightv)
        plightv0 = plightv
        
    if dallasobj and dallasscanned and tdallasconverttime and t > tdallasconverttime + 800:
        ds = [ dallasobj.read_temp(d)  for d in dallasscanned ]
        print(ds)
        for i, t in enumerate(ds):
            if t != 85.0:
                client.publish(topicds[i], "%f"%t)
        tdallasconverttime = 0
    
    if t < t0 + 1000:
        time.sleep_ms(10)
        continue
        
    pled.value(0)
    try:
        humid, temp = SI7021humiditytemp()
        print(humid, temp)
        client.publish(topichumid, "%f"%humid)
        client.publish(topictemp, "%f"%temp)
    except OSError as e:
        print(e)
    pled.value(1)
    
    if dallasobj and t > tdallasconverttime + 800:
        dallasinitconvert()
        tdallasconverttime = t
    t0 = t
    

Sent 119 lines (3789 bytes) to main.py.


In [39]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB8 --baud=115200 
Ready.


In [40]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
boardname    esp8266C

Sent 5 lines (104 bytes) to config.txt.


In [20]:
%sendtofile --source BME280_funcs.py
%sendtofile --source SHT31D_funcs.py

Sent 43 lines (1090 bytes) to SHT31D_funcs.py.


In [32]:
# BME280 humidity sensor code

import machine
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
from BME280_funcs import bme280init, readBME280    #77
from SHT31D_funcs import SHT31Dsetup, readSHT31D   #44



In [28]:
bme280init(i2c)   #77

In [29]:
print(readBME280())

(18.72, 99949.8, 59.0049)


In [23]:
print(i2c.scan())

[68, 119]


In [33]:
SHT31Dsetup(i2c)


b'\x80\x10\xe1' 0xe1
b'\x80pZ' 0x5a


In [34]:
print(readSHT31D())

(20.156, 63.1189)


In [42]:
%websocketconnect --password wpass

** WebSocket connected **
Password: Ready.


In [13]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [14]:
%sendtofile main.py

# SI7021 humidity and dallas string
import time, network, machine, ubinascii, esp

fconfig = dict(x.split()  for x in open("config.txt"))

pled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
plight = machine.ADC(0)

si = network.WLAN()
si.active(True)
si.connect(fconfig["wifiname"], fconfig["wifipassword"])
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print(ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

print("mqtt connecting to", fconfig["boardname"], fconfig["mqttbroker"])
from umqtt.robust import MQTTClient
client = MQTTClient(fconfig["boardname"], fconfig["mqttbroker"])
client.connect()
topicip = fconfig["boardname"]+"/ip"
topicmac = fconfig["boardname"]+"/mac"
topicfreemem = fconfig["boardname"]+"/freemem"
macaddress = ubinascii.hexlify(si.config('mac'),':').decode().upper()

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)


# SI7021 humidity sensor code
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
addrs = i2c.scan()

from BME280_funcs import bme280init, readBME280    #77
from SHT31D_funcs import SHT31Dsetup, readSHT31D   #44

if 0x77 in addrs:
    bme280init(i2c)
else:
    readBME280 = None

if 0x44 in addrs:
    SHT31Dsetup(i2c)
else:
    readSHT31D = None


topicBMEtemp = fconfig["boardname"]+"/BME280/temp"
topicBMEbaro = fconfig["boardname"]+"/BME280/baro"
topicBMEhumid = fconfig["boardname"]+"/BME280/humid"
topicSHTtemp = fconfig["boardname"]+"/SHT31D/temp"
topicSHThumid = fconfig["boardname"]+"/SHT31D/humid"
topicflashled = fconfig["boardname"]+"/flashled"

tdallasconverttime = 0
t0 = time.ticks_ms()
plightv0 = 0

mqttreceived = []
def mqttcallback(topic, msg):
    print((topic, msg))
    while len(mqttreceived) >= 2:
        mqttreceived.pop(0)
    mqttreceived.append((topic, msg))

client.set_callback(mqttcallback)
client.subscribe(topicflashled)
print("Subscribing to", topicflashled)

tipnumbernext = 0
treadBME280next = 0
treadSHT31Dnext = 0

while True:
    time.sleep_ms(10)
    t = time.ticks_ms()
    client.check_msg()
    
    while mqttreceived:
        try:
            flashes = max(1, min(9, int(mqttreceived.pop(0)[1])))
        except Exception as e:
            print(e)
            flashes = 9
        for i in range(flashes):
            pled.value(0)
            time.sleep_ms(5)
            pled.value(1)
            time.sleep_ms(50)
        
    
    if tipnumbernext < t:
        client.publish(topicip, ipnumber, retain=True, qos=1)
        client.publish(topicmac, macaddress, retain=True, qos=1)
        client.publish(topicfreemem, "%d" % esp.freemem(), retain=True, qos=1)
        tipnumbernext = t + 60000
    
    if treadBME280next < t:
        pled.value(0)
        try:
            if readBME280 is not None:
                temp, baro, humid = readBME280()
                print(temp, baro, humid)
                client.publish(topicBMEtemp, "%f"%temp)
                client.publish(topicBMEbaro, "%f"%baro)
                client.publish(topicBMEhumid, "%f"%humid)
        except OSError as e:
            print(e)
        treadBME280next = t + 3000
        pled.value(1)

    if treadSHT31Dnext < t:
        pled.value(0)
        try:
            if readSHT31D is not None:
                temp, humid = readSHT31D()
                print(temp, humid)
                client.publish(topicSHTtemp, "%f"%temp)
                client.publish(topicSHThumid, "%f"%humid)
        except OSError as e:
            print(e)
        pled.value(1)
        treadSHT31Dnext = t + 3030
    



Sent 129 lines (3556 bytes) to main.py.
